# Quantum Rule 90 - experimentos iniciais

In [2]:
# Get packages & Setup IBM account details
!pip install qiskit qiskit_aer qiskit_ibm_runtime numpy pandas  matplotlib pylatexenc

from qiskit_ibm_runtime import QiskitRuntimeService

my_token = "insert TOKEN"
service = QiskitRuntimeService(channel="ibm_quantum", token=my_token)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 31.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 8.1 MB/s eta 0:00:00
  Installing bui

In [3]:
from qiskit_aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime import Session

def simulate_noise(circuit, shots=1024):
    aersim_backend = AerSimulator.from_backend(service.get_backend("ibm_kyoto"))
    pm = generate_preset_pass_manager(backend=aersim_backend, optimization_level=1)
    isa_qc = pm.run(circuit)

    with Session(backend=aersim_backend) as session:
        sampler = Sampler(session=session)
        job = sampler.run([isa_qc], shots=shots)

    return job.result()

def simulate_ideal(circuit, shots=1024):
    aersim_backend = AerSimulator()
    pm = generate_preset_pass_manager(backend=aersim_backend, optimization_level=1)
    isa_qc = pm.run(circuit)

    with Session(backend=aersim_backend) as session:
        sampler = Sampler(session=session)
        job = sampler.run([isa_qc], shots=shots)

    return job.result()

def simulate_real(circuit, shots=1, back=""):
    if back == "":
        backend = service.least_busy(operational=True, simulator=False)
    else:
        backend = service.backend(back)

    # Convert the circuit so it can be ran in the specified backend
    pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
    isa_circuit = pm.run(circuit)

    # Inicia o Sampler e roda o código
    sampler = Sampler(backend=backend)
    job = sampler.run([isa_circuit], shots=shots)
    print(f">>> Job ID: {job.job_id()}")
    print(f">>> Job Status: {job.status()}")
    return job.result()

In [4]:
import qiskit

def prepare_rule90(qc: qiskit.QuantumCircuit):
    qc.h([0,1,2])

    # Middle bit
    qc.cx([0,2,3],[3,3,1])
    qc.barrier()
    qc.reset(3)

    # Left bit
    qc.cx([1,3],[3,2])
    qc.barrier()
    qc.reset(3)

    # Right bit
    qc.cx([1,3],[3,0])
    qc.barrier()
    qc.reset(3)

    qc.h([0,1,2])
    return qc

In [5]:
import pandas as pd

def show_data(result):
    return pd.DataFrame.from_dict(result[0].data.meas.get_counts(), orient="index", columns=['counts'])

In [7]:
circuit = prepare_rule90(qiskit.QuantumCircuit(4,4))

circuit.measure_all()

result_ideal = simulate_ideal(circuit, 1024)
result_noise = simulate_noise(circuit, 1024)

/usr/local/lib/python3.10/dist-packages/qiskit_ibm_runtime/session.py:157: UserWarning: Session is not supported in local testing mode or when using a simulator.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/qiskit_ibm_runtime/session.py:157: UserWarning: Session is not supported in local testing mode or when using a simulator.
  warnings.warn(


In [8]:
df_real1 = show_data(service.job('job-id').result())
df_real2 = show_data(service.job('job-id').result())

In [14]:
df_ideal = show_data(result_ideal)
df_noise = show_data(result_noise)

df = pd.DataFrame(columns=['noise','real','real2','ideal'], dtype='int32')
df[['noise']] = df_noise[['counts']]
df[['real']] = df_real1[['counts']]
df[['real2']] = df_real2[['counts']]
df[['ideal']] = df_ideal[['counts']]
df['ideal'].fillna(value=0, inplace=True)
df

,noise,real,real2,ideal
0101,232,118,169,228.0
0000,231,107,172,262.0
0111,246,101,207,278.0
0010,244,117,172,256.0
0110,13,144,67,0.0
1101,1,4,4,0.0
0011,16,136,75,0.0
1000,2,3,7,0.0
0001,19,140,63,0.0
1110,1,1,1,0.0
